In [1]:
import fitz, csv, re, os
import pandas as pd
import numpy as np

In [6]:
# df1=pd.read_fwf("doc_1.txt")
# df2=pd.read_fwf("doc_2.txt")
fff="C:\\Users\\User\\Desktop\\cta\\patients\\notion\\32fc9e32-5420-4ba6-be1c-9f95c9c6620c.pdf"
txt_fff=os.system(f"ptyhon -m fitz gettext {fff}")
df3=pd.read_fwf(txt_fff)

ValueError: Invalid file path or buffer object type: <class 'int'>

In [42]:
df3

,Szegedi Tudományegyetem,Unnamed: 1
0,Szent-Györgyi Albert Klinikai Központ,NaN
1,064410999 Neurológiai Klinika,NaN
2,6725 Szeged Semmelweis u. 6.,NaN
3,Igazgató: Dr. Klivényi Péter,NaN
4,Tel.: 62/343-400 FAX: 62/545-597,NaN
...,...,...
557,"A zárójelentést õrizze meg, ha ismét kórházba ...",NaN
558,"Kérem, leletével jelentkezzen családi vagy kez...",NaN
559,A zárójelentés 2 példányát átvettem.,NaN
560,e-MedSolution ...,10


In [28]:
np.where(df["Szegedi Tudományegyetem"] == True)

(array([], dtype=int64),)

In [10]:
os.system("python -m fitz gettext test/013360816.pdf")

0

In [67]:
def get_ident(file):
    a={}
    with open(file, encoding="utf8") as f:
        text=f.read()
        taj_pattern=r'\d{3}-\d{3}-\d{3}'
        dob_pattern=r'Szül.dátum:\s+\d{4}.\d{2}.\d{2}'
        adm_date_pattern=r'Felvételi dátum.:\s+\d{4}.\d{2}.\d{2}'
        sex_pattern=r'Születési név...:\s\S+'
        bno_pattern=r'[A-Z]\d{2,}[A-Z]?\d*'
        beav_pattern=r' \d{5} '
        nihss_pattern=r'NI?HSS: \d+'
        mrs_pattern=r'mRS: \d+'
        rr_pattern=r'RR:\s*\d+/\d+'
        pulse_pattern=r'P:\s*\d+'
        

        taj=re.findall(taj_pattern, text)
        taj=taj[0].replace("-","")
        
        dob=re.findall(dob_pattern, text)[0]
        dob=dob.split()[1]

        adm_date=re.findall(adm_date_pattern, text)[0]
        adm_date=adm_date.split(":")[1]

        bno_list=re.findall(bno_pattern, text)

        beav_list=re.findall(beav_pattern, text)
        for i in range(len(beav_list)):
            new_item=beav_list[i].replace(" ","")
            beav_list[i]=new_item
                    
        try:
            nihss=re.findall(nihss_pattern, text)[0]
            nihss=nihss.split()[1]
        except IndexError:
            nihss=None

        try:
            mrs=re.findall(mrs_pattern, text)[0]
            mrs=mrs.split()[1]
        except IndexError:
            mrs=None

        try:
            rr=re.findall(rr_pattern, text)[0]
            rr=rr.split()[1]
        except IndexError:
            rr=None

        try:
            pulse=re.findall(pulse_pattern, text)[0]
            pulse=pulse.split()[1]
        except IndexError:
            pulse=None

        a["Patient_ID"]=taj
        a["DOB"]=dob
        a["Admission_date"]=adm_date
        try:
            sex=re.findall(sex_pattern, text)[0]
            a["Sex"]="female"
        except IndexError:
            a["Sex"]="male"

        if "I48H0" not in bno_list:
            a["AF"]="no"
        else:
            a["AF"]="known/recent"

        if "06042" in beav_list:
            a["Alteplase"]="yes"

        if rr:
            a["RR sys"]=rr.split("/")[0]
            a["RR dia"]=rr.split("/")[1]
        a["P"]=pulse
        a["NIHSS"]=nihss
        a["mRS"]=mrs

    return a


In [68]:
d=get_ident("test\\013360816.txt")
d

{'Patient_ID': '013360816',
 'DOB': '1947.03.07',
 'Admission_date': ' 2021.09.17',
 'Sex': 'male',
 'AF': 'no',
 'P': '84',
 'NIHSS': None,
 'mRS': None}

In [83]:
def get_echo(file):
    regs=[
        r"Echo\w+\s?\(?\d{4}\.?\d{2}?\.?\d{2}?\.?.*",
        r"Ao\w*:?\s+\d*-?\d*-?\d*-?",
        r"B\w+\s?\w*:\s+\d+\s*\w+",
        r"EDD:\s+\d+",
        r"ESD:\s+\d+",
        r"EDV:\s+\d+",
        r"ESV:\s+\d+",
        r"EF:\s+\d+",
        r"IVS:\s+\d+",
        r"PW:\s+\d+",
        r"TAPSE:\s+\d+",
        r"E/?A: \d+/\d+"
    ]
    a={}
    for reg in regs:
        with open(file, encoding="utf8") as f:
            text=f.read()
            match=re.findall(reg, text)[0]
            # print(match)

            dates=re.findall(r'\d{4}\.?\d{2}?\.?\d{2}?',match)
            if dates:
                a["date"]=dates[0]

            # if "Ao" in match:
            #     a["Aorta"]=re.split(r'\s+', match)[1]

            if not "Echo" in match:
                par=match.split(":")[0]
                val=match.split(" ")[-1]
                a[par]=val

    return a

In [84]:
get_echo("patients\\notion\\67efc844-644f-40cf-af5e-632dadb29d78.txt")

{'date': '2021.09.06',
 'Aorta': '21-30-25',
 'Bal pitvar': '42x44x50',
 'EDD': '43',
 'ESD': '28',
 'EDV': '83',
 'ESV': '29',
 'EF': '65',
 'IVS': '10',
 'PW': '10',
 'TAPSE': '20',
 'E/A': '50/71'}

In [28]:
# with open("doc_3.txt", encoding="utf8") as f:
#         text=f.read()
#         taj_pattern=r'\d{3}-\d{3}-\d{3}'
#         taj=re.findall(taj_pattern, text)
#         # taj=taj.replace("-","")
#         print(taj)

['101-441-151']


In [285]:
# def get_lab_param(file, lab_param):
#     a=[]
#     for i in range(file.shape[0]):
#         if lab_param in file.loc[i][0] and "dializált" not in file.loc[i][0]:
#             pattern=r'\s{2,}'
#             p=file.loc[i][0].replace(",",".")
#             p=re.split(pattern,p)
#             par=p[0]
#             val=p[1]
#             # return {i:p}
#             a.append([i,{par:val}])
#     return a

In [16]:
# this is much more straightforward

def get_lab_param(file, lab_param):
    a={}
    l=[]
    for i in range(file.shape[0]):
        text=file.loc[i][0]
        if lab_param in text:
            pattern=r'\s{2,}'
            p=text#.replace(",",".")
            p=re.split(pattern,p)
            # print(p)
            par=p[0]
            try:
                val=p[1]
                l.append(val)
            except IndexError: pass
            # return {i:p}
            # a["no"]=i
            # if "Tromboplasztin" in par:
            #     val=p[0].split(" ")[3]
            a[par]=l
            
    return a

# def get_lab_param(file, lab_param):
#     a={}
#     with open(file, encoding="utf8") as f:
#         text=f.read()
#         if lab_param in text:
#             pattern=r'\s{2,}'
#             p=text#.replace(",",".")
#             p=re.split(pattern,p)
#             par=p[0]
#             val=p[1]
#             # return {i:p}
#             # a["no"]=i
#             a[par]=val
            
#     return a

In [12]:
a={}
for i in range(df3.shape[0]):
    text=df3.loc[i][0]
    # try:
    if "Tromboplasztin" in text:
        pattern=r'\s{2,}'
        p=text#.replace(",",".")
        p=re.split(pattern,p)
        # print(p[1])
        par=p[0]
        val=p[1]
        # return {i:p}
        # a["no"]=i
        if "Tromboplasztin" in par:
            try:
                # val=p[0].split(" ")[3]
                vals=re.findall(r'\d*[,.]?\d*', par)
                for i in vals:
                    if i != "":
                        val=i
            except IndexError: pass
        a[par]=val
    # except IndexError:
    #     continue
a

{'Akt.Parciális Tromboplasztin Idõ 32,3': '32,3'}

In [11]:
df4=pd.read_fwf("test/013360816.txt")

In [33]:
params=[
    "Glükóz",
    "Szenzitív C-reaktív protein",
    "Troponin T (high sensitive)",
    "proBNP  ",
    "Karbamid (Urea)",
    "Kreatinin",
    "eGFR-EPI  ",
    "Fehérvérsejt szám",
    "Protrombin INR",
    "Akt.Parciális Tromboplasztin Idõ",
    "Kvantitatív D-dimer",
    "Haemoglobin A1C",
    "Triglicerid",
    "LDL-koleszterin"
]

result={}
# result.update(get_ident("doc_2.txt"))
ss=[]
for param in params:
    
    # result.update(get_lab_param(df4, param))
    # x=get_lab_param(df4, param)
    # print(x)
    # s=get_lab_param(df4, param)
    # param=param[0:3]
    # s2=pd.Series(s)
    # exec(f"{param}_s = s2")
    # ss.append(f"{param}_s")

# for item in list(result.items()):
#     if "Glükóz" in item[0]:
#         print(item)

C:\Users\User\AppData\Local\Temp\ipykernel_6880\1184593441.py:28: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s2=pd.Series(s)
C:\Users\User\AppData\Local\Temp\ipykernel_6880\1184593441.py:28: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s2=pd.Series(s)
C:\Users\User\AppData\Local\Temp\ipykernel_6880\1184593441.py:28: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s2=pd.Series(s)


In [19]:
rdf=pd.DataFrame(result)

ValueError: All arrays must be of the same length

In [235]:
labs=pd.DataFrame(result, index=[0])
labs["proBNP"]#.to_excel("labs.xlsx")

0    5135,0
Name: proBNP, dtype: object

------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------

In [226]:
cols=[
    "glu_first",
    "CRP_first(mg/l)",
    "TropT(ng/l)",
    "proBNP(pg/ml)",
    "BUN_precontrast(mmol/l)",
    "Creat_precontrast(umol/l)",
    "BUN_postcontrast(mmol/l)",
    "Creat_postcontrast(umol/l)",
    "eGFR(ml/min/tf)",
    "WBC(G/l)",
    "INR",
    "APTI(s)",
    "D-dimer(ug/ml)",
    "HgbA1c(mmol/mol)",
    "HgbA1c(%)",
    "Triglicerid(mmol/l)",
    "LDL-chol(mmol/l)"
]

labs=pd.DataFrame(columns=params)
labs

,Glükóz,C-reaktív protein,Troponin T,proBNP,Urea,Kreatinin,eGFR,Fehérvérsejt,INR,Akt.Parciális Tromboplasztin Idõ,D-dimer,Haemoglobin A1C,Triglicerid,LDL-koleszterin


In [341]:
l=[]
for i in range(len(params)):
    if result[i]:
        dict=result[i][0][1]
        l.append(dict)
l

[{'Glükóz': '11.5'},
 {'Szenzitív C-reaktív protein': '15.7'},
 {'Troponin T (high sensitive)': '20'},
 {'proBNP': '4668.0'},
 {'Karbamid (Urea)': '11.5'},
 {'Kreatinin': '147'},
 {'eGFR-EPI': '29'},
 {'Fehérvérsejt szám': '10.86'},
 {'Protrombin INR': '1.19'},
 {'Akt.Parciális Tromboplasztin Idõ 39.4': 's'},
 {'Kvantitatív D-dimer': '2.25'},
 {'Haemoglobin A1C': '7.5'},
 {'Triglicerid': '1.08'},
 {'LDL-koleszterin': '2.79'}]

In [336]:
labs.loc[0]=

,Glükóz,C-reaktív protein,Troponin T,proBNP,Urea,Kreatinin,eGFR,Fehérvérsejt,INR,Akt.Parciális Tromboplasztin Idõ,D-dimer,Haemoglobin A1C,Triglicerid,LDL-koleszterin
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.79


In [346]:
# l[0]["Glükóz"]
d={}
try:
    for i in range(len(params)):
        for param in params:
            value=l[i][param]
            print(param,value)
            d[param]=value
            i+=1
except KeyError:
    i+=1

Glükóz 11.5
Szenzitív C-reaktív protein 15.7
Troponin T (high sensitive) 20
proBNP 4668.0
Karbamid (Urea) 11.5
Kreatinin 147
eGFR-EPI 29
Fehérvérsejt szám 10.86
Protrombin INR 1.19


In [349]:
pd.DataFrame(d, index=[0])

,Glükóz,Szenzitív C-reaktív protein,Troponin T (high sensitive),proBNP,Karbamid (Urea),Kreatinin,eGFR-EPI,Fehérvérsejt szám,Protrombin INR
0,11.5,15.7,20,4668.0,11.5,147,29,10.86,1.19


In [83]:
with fitz.open("doc_3.pdf") as doc:
    print(doc.metadata)
    # for key in doc.xref_get_keys(xref=1):
    #     print(key, "=" , doc.xref_get_key(xref=1, key=key))
    # xref=doc.xref_stream_raw(xref=1)
    

# print(xref)

{'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'ISH Medsolution report generator.', 'creationDate': '', 'modDate': '', 'trapped': '', 'encryption': None}


In [178]:
import glob
path = os.getcwd()
for txt in glob.glob(f"{path}/*.txt"):
    print(txt.split("\\")[-1].split(".")[0])

doc_1
doc_2
doc_3


In [ ]:
import requests

x = requests.get('http://http://ish.szote.u-szeged.hu/ems2', timeout=3)
print(x.status_code)

In [193]:
with open("doc_2.txt", encoding="utf8") as f:
    text=f.read()
    bno_pattern=r'[A-Z]\d{2,}[A-Z]?\d*'
    print(re.findall(bno_pattern, text))

['I6390', 'I6600', 'I6720', 'I48H0', 'I2520', 'E1080', 'I10H0', 'E7800', 'Z2980']


In [9]:
df = pd.read_csv("test/2021_09_2022_07.csv")
taj=df["TAJ"].unique()
l=pd.DataFrame(taj)
l.to_excel("unique")

,0
0,013032157
1,015877493
2,013757030
3,076931051
4,056132292
...,...
880,063061817
881,102965744
882,063447196
883,102884265


In [15]:
import pydicom as dicom

ds = dicom.read_file("test/S0000001/O0000001")
ds.StudyDate

'20220317'

In [10]:
df = pd.read_excel("Stroke_CTA_v1.xlsx")
df.head()

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,No.,Patient_ID,DOB,Admission_date,Sex,Age(yrs),Prev_stroke/TIA/thromboembolia,AF(known/new),pangásos SZE / kard.dekomp.,vascular disease,...,Ao_insuff,Pulm_insuff,Pulm_pressure,Mitr_insuff,Tricusp_insuff,Hemorrh_post,Emissio,Cause_of_death,NIHSS_emiss,Comment
0,1,59161963,1942-05-28,2022-07-23,female,80.0,no,recent,no,no,...,jelzett,NaN,50,II,II-III,hemorrh_transf,other_hosp,NaN,17.0,"mélyvénába belógó, mutatóujj nagyságú thrombus..."
1,2,19310534,1955-01-14,2022-03-07,male,67.0,TIA,no,no,no,...,II-III,kicsi,48+cvp,jelzett,I-II,NaN,home,NaN,NaN,"PD, chr WM laesiok"
2,3,26642983,1966-04-08,2022-03-25,male,55.0,kmn,no,no,no,...,NaN,NaN,NaN,NaN,NaN,NaN,home,NaN,0.0,"PFO, paradox embolizáció, Leiden és protrombin..."
3,4,59230988,1942-07-09,2022-03-28,female,79.0,TIA,known,yes,yes,...,I,közepes,33+15,I-II,I,NaN,home,NaN,NaN,bal ACI plakk/thrombus
4,5,101441151,1933-06-15,2022-03-29,female,88.0,kmn,no,yes,yes,...,II,NaN,26+cvp,III,I,NaN,died,other,NaN,agyi hypoperfusio


In [6]:
df["AF(known/new)"].describe()

count     86
unique     3
top       no
freq      68
Name: AF(known/new), dtype: object

In [7]:
df["Age(yrs)"].describe()

count    86.000000
mean     71.383721
std      11.984469
min      40.000000
25%      65.000000
50%      72.500000
75%      79.750000
max      92.000000
Name: Age(yrs), dtype: float64

In [8]:
df["Sex"].describe()

count         86
unique         2
top       female
freq          51
Name: Sex, dtype: object

In [13]:
df["LAA thrombus"].value_counts()

no                     54
ambiguous              23
yes?                    3
no?                     2
yes                     1
22 jan. mellkas CTA     1
Name: LAA thrombus, dtype: int64